In [1]:
import numpy as np
from astropy.io import fits
from astropy.coordinates import SkyCoord
import astropy.units as u
import matplotlib.pyplot as plt
import findsources as find
plt.style.use('mplstyle')

In [2]:
from astropy.io import fits
image = 'final_mosaic.fits'  # Change to your file
# image = 'final_mosaicadaptive.fits'  # Change to your file
with fits.open(image) as hdul:
    beam_size = hdul[0].header['BMAJ']* 3600
    print(beam_size)  # Look for BMAJ, BMIN, BPA


40.00000003092


In [3]:
target_ra_hms = '02h00m12s'
target_dec_dms = '-30d53m24s'
search_radius = 20 * 3600 * u.arcsec
flux_thresh = 1. # Jy
# Set matching radius (e.g., 45 arcseconds for GLEAM-X resolution)
match_radius = beam_size * u.arcsec 
match_radius = 40 * u.arcsec # sqrt(40*22)
si = -0.8

# Load GLEAM catalog
gleam_catalog = "GLEAM_EGC_v2.fits.gz"
# gleamx1_catalog = "VIII_110_catalog.dat.gz.fits"
# gleamx1_catalog = "VIII_113_catalog2.dat.gz.fits"
gleamx1_catalog = "VIII_113_catalog2glx.dat.gz.fits" # a modified version that gives alpha values
tgss_catalog = "TGSSADR1_7sigma_catalog.fits"
tgss_catalog = "J_A+A_598_A78_table3.dat.gz.fits.gz"
pybdsf_catalog= "final_mosaic.pybdsf.srl.fits"
# pybdsf_catalog= "final_mosaicadaptive.pybdsf.srl.fits"
# nvss_catalog= "nvss_catalog.fits"
nvss_catalog= "VIII_65_nvss.dat.gz.fits.gz"


In [4]:
# with fits.open(gleamx1_catalog, mode="update") as hdul:
#     for i, col in enumerate(hdul[1].columns, start=1):
#         if col.name == 'alpha-SP':
#             print(f"Column index is {i}")


In [5]:
# with fits.open(gleamx2_catalog, mode="update") as hdul:
#     hdr = hdul[1].header
#     hdr['TNULL384'] = '---'
#     hdul.flush()  # Save changes


In [6]:
# with fits.open(gleamx1_catalog) as hdul:
#     print(hdul[1].columns)   # RA in degrees

In [7]:
nu_reference = 147.4
nu_gleam = 151 
nu_gleam_x1 = 151 
# nu_gleam_x1 = 151 
nu_nvss = 1400 

fac_gleam = (nu_reference/nu_gleam)**si
print(fac_gleam)

fac_gleam_x1 = (nu_reference/nu_gleam_x1)**si
print(fac_gleam_x1)

fac_nvss = (nu_reference/nu_nvss)**si
print(fac_nvss)

1.019491410356563
1.019491410356563
6.054864682394719


In [8]:
def matched_sources(gleam_catalog, pybdsf_catalog, flux_thresh, target_ra_hms, target_dec_dms, search_radius, match_radius, names):
    
    gleam_ra, gleam_dec, gleam_flux, gleam_flux_err, gleam_flux_151, gleam_flux_151_err, alpha, alpha_err, name = find.find_sources_spind(gleam_catalog, flux_thresh, target_ra_hms, target_dec_dms, search_radius, names[0])
    pybdsf_ra, pybdsf_dec, pybdsf_flux, pybdsf_flux_err = find.find_sources(pybdsf_catalog, flux_thresh, target_ra_hms, target_dec_dms, search_radius, names[1])
    
    # argsort = np.argsort(-gleam_flux_151)
    # print(gleam_flux_151[argsort][:10])
    
    # Convert to SkyCoord objects
    pybdsf_coords = SkyCoord(ra=pybdsf_ra * u.deg, dec=pybdsf_dec * u.deg)
    gleam_coords = SkyCoord(ra=gleam_ra * u.deg, dec=gleam_dec * u.deg)
    
    # Find nearest neighbor in GLEAM catalog for each PyBDSF source
    idx_gleam, d2d, _ = pybdsf_coords.match_to_catalog_sky(gleam_coords)
    
    # Set matching radius (e.g., 45 arcseconds for GLEAM-X resolution)
    matched_mask = d2d < match_radius*2
    
    # Apply mask to get matched indices
    matched_pybdsf = np.where(matched_mask)[0]  # Indices in PyBDSF catalog
    matched_gleam = idx_gleam[matched_mask]      # Nearest neighbor indices in GLEAM catalog
    matched_sep = d2d[matched_mask].to(u.arcsec) # Separation in arcsec
    return pybdsf_ra[matched_pybdsf], pybdsf_dec[matched_pybdsf], gleam_ra[matched_gleam], \
    gleam_dec[matched_gleam], pybdsf_flux[matched_pybdsf], gleam_flux[matched_gleam], \
    pybdsf_flux_err[matched_pybdsf], gleam_flux_err[matched_gleam], gleam_flux_151[matched_gleam], gleam_flux_151_err[matched_gleam],\
    alpha[matched_gleam], alpha_err[matched_gleam], name[matched_gleam]

In [9]:
names = ['gleam', 'pybdsf']

pybdsf_ra, pybdsf_dec, gleam_ra, gleam_dec, pybdsf_flux, gleam_flux, pybdsf_flux_err, gleam_flux_err, gleam_flux_151, gleam_flux_151_err, alpha, alpha_err, name = matched_sources(gleam_catalog, pybdsf_catalog, flux_thresh,
                                                                                                                       target_ra_hms, target_dec_dms,
                                                                                                                       search_radius, match_radius, 
                                                                                                                       names)
gleam_flux *= (fac_gleam)
gleam_flux_err *= (fac_gleam)

In [10]:
argsort = np.argsort(-pybdsf_flux)

print(pybdsf_flux[argsort][:10], pybdsf_flux_err[argsort][:10],)
print()
# print(gleam_flux[argsort][:10], gleam_flux_err[argsort][:10],)
print(gleam_flux_151[argsort][:10], gleam_flux_151_err[argsort][:10])

print(alpha[argsort][:10],)
print(alpha_err[argsort][:10],)
print(name[argsort][:10])



[19.07944087 10.08407524  6.2040596   4.66649829  4.07043437  2.63676615
  2.59285157  2.32029264  2.01481443  2.00971007] [0.03169417 0.1228052  0.07585994 0.01961611 0.04961429 0.05010357
 0.01047804 0.02858712 0.01883139 0.04687232]

[17.949156  16.595154  16.595154   4.6637244  3.94791    2.476469
  2.6904912  2.1443262  1.8060807  1.8912518] [0.02400285 0.02889125 0.02889125 0.02368167 0.01800959 0.02056924
 0.01968022 0.01649848 0.01746771 0.02104062]
[-0.862534   -0.80219555 -0.80219555 -0.7282266  -0.59387106 -0.89094305
 -1.1502343  -0.75958085 -0.58621854 -0.6760057 ]
[0.0136427  0.01366115 0.01366115 0.01435938 0.01448209 0.01516625
 0.01472999 0.01566539 0.0171046  0.01651383]
['GLEAM J020012-305324' 'GLEAM J015035-293158' 'GLEAM J015035-293158'
 'GLEAM J015200-294056' 'GLEAM J021527-312144' 'GLEAM J014930-283819'
 'GLEAM J020820-303928' 'GLEAM J020620-332547' 'GLEAM J020311-334722'
 'GLEAM J015853-273607']


In [11]:
print(name[argsort][:10])
print(pybdsf_ra[argsort][:10])
print(pybdsf_dec[argsort][:10])


['GLEAM J020012-305324' 'GLEAM J015035-293158' 'GLEAM J015035-293158'
 'GLEAM J015200-294056' 'GLEAM J021527-312144' 'GLEAM J014930-283819'
 'GLEAM J020820-303928' 'GLEAM J020620-332547' 'GLEAM J020311-334722'
 'GLEAM J015853-273607']
[30.05078099 27.64298912 27.65874067 28.00432669 33.86338507 27.37599285
 32.08494319 31.58750571 30.80088555 29.72171712]
[-30.89120325 -29.54349408 -29.51968    -29.68403569 -31.36275502
 -28.64091047 -30.65952121 -33.4306934  -33.79015013 -27.60315898]


In [12]:
names = ['gleamx1', 'pybdsf']

pybdsf_ra_x1, pybdsf_dec_x1, gleamx1_ra, gleamx1_dec, pybdsf_flux_x1, gleamx1_flux, pybdsf_flux_x1_err, gleamx1_flux_err, gleamx1_flux_151, gleamx1_flux_151_err, alphax1, alphax1_err, name = matched_sources(gleamx1_catalog, pybdsf_catalog, flux_thresh,
                                                                                                                       target_ra_hms, target_dec_dms,
                                                                                                                       search_radius, match_radius, 
                                                                                                                       names)

gleamx1_flux *= (fac_gleam_x1)
gleamx1_flux_err *= (fac_gleam_x1)

In [13]:
argsortx1 = np.argsort(-pybdsf_flux_x1)

print(pybdsf_flux_x1[argsortx1][:10], pybdsf_flux_x1_err[argsortx1][:10],)
print()
# print(gleamx1_flux[argsortx1][:10], gleamx1_flux_err[argsortx1][:10],)
print(gleamx1_flux_151[argsortx1][:10], gleamx1_flux_151_err[argsortx1][:10],)

print('sp index')
print(alphax1[argsort][:10],)
print(alphax1_err[argsort][:10],)
print()

print(name[argsort][:10])


[19.07944087 10.08407524  6.2040596   4.66649829  4.07043437  2.63676615
  2.59285157  2.32029264  2.01481443  2.00971007] [0.03169417 0.1228052  0.07585994 0.01961611 0.04961429 0.05010357
 0.01047804 0.02858712 0.01883139 0.04687232]

[18.8636    9.360348  7.394917  4.915171  3.96779   2.58331   2.944448
  2.400819  1.877183  1.923148] [ 0.003432  0.003294  0.003461  0.002851 -1.        0.002428  0.002628
  0.002273  0.002385  0.002359]
sp index
[-0.7756978   0.          0.         -0.69244562 -0.60566428 -0.81505837
 -1.07143745 -0.72961838 -0.60930943 -0.68338672]
[0.01354298 0.         0.         0.01358501 0.01352334 0.01354934
 0.01359471 0.01357561 0.01356063 0.01358604]

['GLEAM-X J020012.1-305326' 'GLEAM-X J015034.3-293239'
 'GLEAM-X J015038.1-293109' 'GLEAM-X J015201.0-294100'
 'GLEAM-X J021527.3-312147' 'GLEAM-X J014930.3-283825'
 'GLEAM-X J020820.4-303934' 'GLEAM-X J020620.9-332549'
 'GLEAM-X J020312.1-334724' 'GLEAM-X J015853.1-273609']


In [14]:
pybdsf_ra_x1

array([33.86338507, 33.00824725, 32.08494319, 31.58750571, 30.80088555,
       30.11608792, 30.05078099, 29.72171712, 28.4286545 , 28.34344057,
       28.16570967, 28.00432669, 27.64298912, 27.65874067, 27.37599285,
       26.92796656], dtype='>f8')

In [15]:
['GLEAM-X J020012.1-305326' 'GLEAM-X J015201.0-294100' 'GLEAM-X J021527.3-312147' 'GLEAM-X J015034.3-293239' 'GLEAM-X J020820.4-303934' 'GLEAM-X J020620.9-332549'
 'GLEAM-X J020312.1-334724' 'GLEAM-X J015853.1-273609' 'GLEAM-X J021202.0-314950' 'GLEAM-X J015322.5-311146']
['GLEAM J020012-305324' 'GLEAM J015200-294056' 'GLEAM J021527-312144' 'GLEAM J014930-283819' 'GLEAM J020820-303928' 'GLEAM J020620-332547' 
 'GLEAM J020311-334722' 'GLEAM J015853-273607' 'GLEAM J021201-314946' 'GLEAM J015322-311145']

['GLEAM J020012-305324GLEAM J015200-294056GLEAM J021527-312144GLEAM J014930-283819GLEAM J020820-303928GLEAM J020620-332547GLEAM J020311-334722GLEAM J015853-273607GLEAM J021201-314946GLEAM J015322-311145']

In [16]:
def matched_sources_tgss(gleam_catalog, pybdsf_catalog, flux_thresh, target_ra_hms, target_dec_dms, search_radius, match_radius, names):
    
    gleam_ra, gleam_dec, gleam_flux, gleam_flux_err = find.find_sources(gleam_catalog, flux_thresh, target_ra_hms, target_dec_dms, search_radius, names[0])
    pybdsf_ra, pybdsf_dec, pybdsf_flux, pybdsf_flux_err = find.find_sources(pybdsf_catalog, flux_thresh, target_ra_hms, target_dec_dms, search_radius, names[1])
    
    
    # Convert to SkyCoord objects
    pybdsf_coords = SkyCoord(ra=pybdsf_ra * u.deg, dec=pybdsf_dec * u.deg)
    gleam_coords = SkyCoord(ra=gleam_ra * u.deg, dec=gleam_dec * u.deg)
    
    # Find nearest neighbor in GLEAM catalog for each PyBDSF source
    idx_gleam, d2d, _ = pybdsf_coords.match_to_catalog_sky(gleam_coords)
    
    # Set matching radius (e.g., 45 arcseconds for GLEAM-X resolution)
    matched_mask = d2d < match_radius
    
    # Apply mask to get matched indices
    matched_pybdsf = np.where(matched_mask)[0]  # Indices in PyBDSF catalog
    matched_gleam = idx_gleam[matched_mask]      # Nearest neighbor indices in GLEAM catalog
    matched_sep = d2d[matched_mask].to(u.arcsec) # Separation in arcsec
    return pybdsf_ra[matched_pybdsf], pybdsf_dec[matched_pybdsf], gleam_ra[matched_gleam], gleam_dec[matched_gleam], pybdsf_flux[matched_pybdsf], gleam_flux[matched_gleam], pybdsf_flux_err[matched_pybdsf], gleam_flux_err[matched_gleam]

In [17]:
names = ['tgss', 'pybdsf']

pybdsf_ra1, pybdsf_dec1, tgss_ra, tgss_dec, pybdsf_flux1, tgss_flux, pybdsf_flux1_err, tgss_flux_err = matched_sources_tgss(tgss_catalog, pybdsf_catalog, flux_thresh,
                                                                                                                       target_ra_hms, target_dec_dms,
                                                                                                                       search_radius, match_radius*1.5, 
                                                                                                                       names)

tgss_flux *= 1e-3 # converted to Jy
tgss_flux_err *= 1e-3 # converted to Jy

In [18]:
argsort1 = np.argsort(-pybdsf_flux1)
print(pybdsf_flux1[argsort1][:10], pybdsf_flux1_err[argsort1][:10],)
print()
print(tgss_flux[argsort1][:10], tgss_flux_err[argsort1][:10],)

[19.07944087 10.08407524  6.2040596   4.66649829  4.07043437  2.63676615
  2.59285157  2.32029264  2.01481443  2.00971007] [0.03169417 0.1228052  0.07585994 0.01961611 0.04961429 0.05010357
 0.01047804 0.02858712 0.01883139 0.04687232]

[22.3092 11.8522 11.8522  4.4188  3.8508  2.4844  3.0099  2.0635  1.9306
  1.6984] [2.2311 1.1857 1.1857 0.4423 0.3851 0.249  0.3012 0.2064 0.1931 0.17  ]


In [19]:
argsort1 = np.argsort(-pybdsf_flux1)
print(pybdsf_flux1[argsort1][:10], pybdsf_flux1_err[argsort1][:10],)
print()
print(tgss_flux[argsort1][:10], tgss_flux_err[argsort1][:10],)

[19.07944087 10.08407524  6.2040596   4.66649829  4.07043437  2.63676615
  2.59285157  2.32029264  2.01481443  2.00971007] [0.03169417 0.1228052  0.07585994 0.01961611 0.04961429 0.05010357
 0.01047804 0.02858712 0.01883139 0.04687232]

[22.3092 11.8522 11.8522  4.4188  3.8508  2.4844  3.0099  2.0635  1.9306
  1.6984] [2.2311 1.1857 1.1857 0.4423 0.3851 0.249  0.3012 0.2064 0.1931 0.17  ]


In [20]:
# names = ['nvss', 'pybdsf']
# # names = ['tgss', 'pybdsf']

# pybdsf_ra2, pybdsf_dec2, nvss_ra, nvss_dec, pybdsf_flux2, nvss_flux, pybdsf_flux2_err, nvss_flux_err = matched_sources(nvss_catalog, pybdsf_catalog, flux_thresh/1000,
#                                                                                                                        target_ra_hms, target_dec_dms,
#                                                                                                                        search_radius, match_radius, 
#                                                                                                                        names)
# # nvss_flux *= (fac_nvss*1e-3)
# # nvss_flux_err *= (fac_nvss*1e-3)
# fac_nvss = 1
# print(fac_nvss)
# nvss_flux *= (fac_nvss)
# nvss_flux_err *= (fac_nvss)


In [21]:
# argsort2 = np.argsort(-pybdsf_flux2)
# print(pybdsf_flux2[argsort2][:10], pybdsf_flux2_err[argsort2][:10],)
# print()
# print(nvss_flux[argsort2][:10], nvss_flux_err[argsort2][:10],)

In [22]:
# nu_nvss = 1.4e3 # MHz
# nu_pybdsf = 147.4 # MHz

# alpha = ( np.log(nvss_flux[argsort2][:10]) - np.log(pybdsf_flux2_err[argsort2][:10]) ) / ( np.log(nu_nvss) - np.log(nu_pybdsf) ) 

In [23]:
# alpha

# Spectral Index

In [24]:
def matched_sources_nvss(gleam_catalog, pybdsf_catalog, flux_thresh, target_ra_hms, target_dec_dms, search_radius, match_radius, names):
    
    gleam_ra, gleam_dec, gleam_flux, gleam_flux_err = find.find_sources(gleam_catalog, flux_thresh, target_ra_hms, target_dec_dms, search_radius, names[0])
    pybdsf_ra, pybdsf_dec, pybdsf_flux, pybdsf_flux_err = find.find_sources(pybdsf_catalog, flux_thresh, target_ra_hms, target_dec_dms, search_radius, names[1])
    
    # argsort = np.argsort(-gleam_flux_151)
    # print(gleam_flux_151[argsort][:10])
    
    # Convert to SkyCoord objects
    pybdsf_coords = SkyCoord(ra=pybdsf_ra * u.deg, dec=pybdsf_dec * u.deg)
    gleam_coords = SkyCoord(ra=gleam_ra * u.deg, dec=gleam_dec * u.deg)
    
    # Find nearest neighbor in GLEAM catalog for each PyBDSF source
    idx_gleam, d2d, _ = pybdsf_coords.match_to_catalog_sky(gleam_coords)
    
    # Set matching radius (e.g., 45 arcseconds for GLEAM-X resolution)
    matched_mask = d2d < match_radius*2
    
    # Apply mask to get matched indices
    matched_pybdsf = np.where(matched_mask)[0]  # Indices in PyBDSF catalog
    matched_gleam = idx_gleam[matched_mask]      # Nearest neighbor indices in GLEAM catalog
    matched_sep = d2d[matched_mask].to(u.arcsec) # Separation in arcsec
    return pybdsf_ra[matched_pybdsf], pybdsf_dec[matched_pybdsf], gleam_ra[matched_gleam], \
    gleam_dec[matched_gleam], pybdsf_flux[matched_pybdsf], gleam_flux[matched_gleam], \
    pybdsf_flux_err[matched_pybdsf], gleam_flux_err[matched_gleam]

In [25]:
names = ['nvss', 'pybdsf']
search_radius = 20 * 3600 * u.arcsec
# flux_thresh = 2 # Jy
# # Set matching radius (e.g., 45 arcseconds for GLEAM-X resolution)
# match_radius = beam_size * u.arcsec 
# match_radius = 30 * u.arcsec # sqrt(40*22)

pybdsf_ra2, pybdsf_dec2, nvss_ra, nvss_dec, pybdsf_flux2, nvss_flux, pybdsf_flux2_err, nvss_flux_err = matched_sources_nvss(nvss_catalog, pybdsf_catalog, flux_thresh*1e-5,
                                                                                                                       target_ra_hms, target_dec_dms,
                                                                                                                       search_radius, match_radius, 
                                                                                                                       names)
fac_nvss = 1. 
nvss_flux *= (fac_nvss)
nvss_flux_err *= (fac_nvss)


In [26]:
nu_nvss = 1.4e3 # MHz
nu_pybdsf = 147.4 # MHz

In [27]:
pybdsf_ra2.shape, nvss_ra.shape

((620,), (620,))

In [28]:
argsort2 = np.argsort(-pybdsf_flux2)
print(pybdsf_flux2[argsort2][:10], pybdsf_flux2_err[argsort2][:10],)
print()
print(nvss_flux[argsort2][:10], nvss_flux_err[argsort2][:10],)

[19.07944087 10.08407524  6.2040596   4.66649829  4.07043437  2.63676615
  2.59285157  2.32029264  2.01481443  2.00971007] [0.03169417 0.1228052  0.07585994 0.01961611 0.04961429 0.05010357
 0.01047804 0.02858712 0.01883139 0.04687232]

[3.7516 1.6939 1.2806 1.0166 0.9762 0.4396 0.3455 0.4168 0.5374 0.4374] [0.13   0.0558 0.0469 0.0356 0.0293 0.0155 0.012  0.0125 0.0189 0.0131]


In [29]:
alpha = ( np.log(nvss_flux) - np.log(pybdsf_flux2) ) / ( np.log(nu_nvss) - np.log(nu_pybdsf) ) 

In [30]:
alpha[argsort2][:10]

array([-0.7225113 , -0.79247555, -0.70094215, -0.67698478, -0.63429069,
       -0.79581589, -0.89535827, -0.76267569, -0.58706982, -0.67740787])

In [31]:

print('GMRT - GLEAM 154 MHz [flux][error]')

print(pybdsf_flux1[argsort][:10], pybdsf_flux1_err[argsort][:10],)
print()
print(gleam_flux[argsort][:10], gleam_flux_err[argsort][:10],)

print()

print('GMRT - GLEAM-X 200 MHz [flux][error]')
print(pybdsf_flux_x1[argsortx1][:10], pybdsf_flux_x1_err[argsortx1][:10],)
print()
print(gleamx1_flux[argsortx1][:10], gleamx1_flux_err[argsortx1][:10],)

print()

print('GMRT - TGSS [flux][error]')
print(pybdsf_flux1[argsort1][:10], pybdsf_flux1_err[argsort1][:10],)
print()
print(tgss_flux[argsort1][:10], tgss_flux_err[argsort1][:10],)


print()


print('GMRT - NVSS [flux][error] and sp ind. ')
print(pybdsf_flux2[argsort2][:10], pybdsf_flux2_err[argsort2][:10],)
print()
print(nvss_flux[argsort2][:10], nvss_flux_err[argsort2][:10],)
print()
print(alpha[argsort2][:10])

GMRT - GLEAM 154 MHz [flux][error]
[ 1.06167681 19.07944087  2.00971007  1.34869239  4.07043437  1.01728599
  1.20893383  1.08517233  2.59285157  1.02766182] [0.01287127 0.03169417 0.04687232 0.01061742 0.04961429 0.01185612
 0.03034815 0.02174727 0.01047804 0.01118643]

[15.118112  13.500846  13.500846   3.9354248  3.5222824  2.0855205
  2.0953486  1.8133367  1.6092044  1.647975 ] [0.01140577 0.01558354 0.01558354 0.01113222 0.00823695 0.00672381
 0.00940179 0.00725005 0.00805688 0.00737733]

GMRT - GLEAM-X 200 MHz [flux][error]
[19.07944087 10.08407524  6.2040596   4.66649829  4.07043437  2.63676615
  2.59285157  2.32029264  2.01481443  2.00971007] [0.03169417 0.1228052  0.07585994 0.01961611 0.04961429 0.05010357
 0.01047804 0.02858712 0.01883139 0.04687232]

[15.87564462  7.9299814   6.10610719  4.10427056  3.46922018  2.10127171
  2.15403243  1.99186499  1.64988271  1.64203568] [0.00172906 0.00150069 0.00150885 0.00109595 0.00090633 0.00094915
 0.00098381 0.00094099 0.00095424 0.0